<a href="https://colab.research.google.com/github/priyeshroy10155/computervision/blob/main/Binary_class_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Colab Notebooks/cat and dog')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['dog', 'cat']
Types of classes labels found:  2


In [ ]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/Colab Notebooks/cat and dog' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

In [ ]:
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

In [ ]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  320
dog    160
cat    160
Name: Labels, dtype: int64


In [ ]:
import cv2
path = '/content/drive/MyDrive/Colab Notebooks/cat and dog/'
dataset_path = os.listdir('/content/drive/MyDrive/Colab Notebooks/cat and dog')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [ ]:
images

In [ ]:
#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255]. 

images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(320, 224, 224, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

In [ ]:
y=y.reshape(-1,1)
y

In [ ]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[-5:])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(304, 224, 224, 3)
(304, 2)
(16, 224, 224, 3)
(16, 2)


In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.05, random_state=415)
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(288, 224, 224, 3)
(288, 2)
(16, 224, 224, 3)
(16, 2)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization

In [ ]:
# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())
# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Output Layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 activation_4 (Activation)   (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 batch_normalization_2 (Batc  (None, 27, 27, 96)       384       
 hNormalization)                                                 
                                                                 
 conv2d_11 (Conv2D)          (None, 17, 17, 256)       2973952   
                                                                 
 activation_5 (Activation)   (None, 17, 17, 256)      

In [ ]:
# (4) Compile 
optimizers = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer= optimizers,\
 metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True
)

In [ ]:
hist = model.fit(train_x, train_y, validation_data=(val_x,val_y),epochs=40, verbose=2,callbacks=[callback])

Epoch 1/40
9/9 - 1s - loss: 31.1795 - accuracy: 0.5035 - val_loss: 129.5801 - val_accuracy: 0.5000 - 1s/epoch - 157ms/step
Epoch 2/40
9/9 - 0s - loss: 15.5827 - accuracy: 0.5417 - val_loss: 172.0660 - val_accuracy: 0.5000 - 393ms/epoch - 44ms/step
Epoch 3/40
9/9 - 1s - loss: 9.2611 - accuracy: 0.6285 - val_loss: 49.5690 - val_accuracy: 0.5000 - 551ms/epoch - 61ms/step
Epoch 4/40
9/9 - 0s - loss: 5.6570 - accuracy: 0.6250 - val_loss: 56.8514 - val_accuracy: 0.5000 - 395ms/epoch - 44ms/step
Epoch 5/40
9/9 - 1s - loss: 2.3433 - accuracy: 0.7118 - val_loss: 27.9371 - val_accuracy: 0.4375 - 536ms/epoch - 60ms/step
Epoch 6/40
9/9 - 1s - loss: 1.7923 - accuracy: 0.7708 - val_loss: 15.3820 - val_accuracy: 0.5000 - 534ms/epoch - 59ms/step
Epoch 7/40
9/9 - 1s - loss: 1.3718 - accuracy: 0.7292 - val_loss: 3.7193 - val_accuracy: 0.6875 - 530ms/epoch - 59ms/step
Epoch 8/40
9/9 - 0s - loss: 0.9003 - accuracy: 0.8507 - val_loss: 10.4958 - val_accuracy: 0.4375 - 395ms/epoch - 44ms/step
Epoch 9/40
9/9 

In [ ]:
model.evaluate(test_x,test_y)

1/1 [==============================] - 0s 33ms/step - loss: 4.2123 - accuracy: 0.5000


[4.2123003005981445, 0.5]

# with augmentor

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.python import resource_loader
datagen = ImageDataGenerator(rotation_range = 40,     width_shift_range=0.6,
    height_shift_range=0.5, shear_range=0.0,
    zoom_range=0.0, validation_split=0.2, rescale = 1/255)

In [ ]:
train_generator = datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/cat and dog',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary', subset = 'training')

Found 256 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/cat and dog',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary', subset = 'validation')

Found 64 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True
)

In [ ]:
hist = model.fit(train_generator, validation_data=val_generator,epochs=40, verbose=2, callbacks=[callback])

Epoch 1/40
8/8 - 6s - loss: 0.7120 - acc: 0.4922 - val_loss: 0.6946 - val_acc: 0.5312 - 6s/epoch - 790ms/step
Epoch 2/40
8/8 - 5s - loss: 0.6991 - acc: 0.4180 - val_loss: 0.6898 - val_acc: 0.5469 - 5s/epoch - 570ms/step
Epoch 3/40
8/8 - 5s - loss: 0.6922 - acc: 0.5039 - val_loss: 0.7064 - val_acc: 0.5000 - 5s/epoch - 571ms/step
Epoch 4/40
8/8 - 5s - loss: 0.6982 - acc: 0.5039 - val_loss: 0.6911 - val_acc: 0.5000 - 5s/epoch - 573ms/step
Epoch 5/40
8/8 - 5s - loss: 0.6948 - acc: 0.4883 - val_loss: 0.6917 - val_acc: 0.5000 - 5s/epoch - 571ms/step
Epoch 6/40
8/8 - 5s - loss: 0.6925 - acc: 0.5156 - val_loss: 0.6922 - val_acc: 0.5156 - 5s/epoch - 570ms/step
Epoch 7/40
8/8 - 5s - loss: 0.6900 - acc: 0.5195 - val_loss: 0.6948 - val_acc: 0.5000 - 5s/epoch - 576ms/step


In [ ]:
model.evaluate(val_generator)

2/2 [==============================] - 1s 458ms/step - loss: 0.6913 - acc: 0.5781


[0.6912565231323242, 0.578125]

## After data augmentation accuracy increased by 7%